# Deep Learning

## Assignment 4

Previously in 2_ fullyconnected.ipynb and 3_regularization, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html)

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:

* convolutions need the image data formatted as a cube(width by height by #channels)
* lables as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
           / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) 
        return tf.matmul(hidden, layer4_weights) + layer4_biases
        
    #  Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.451322
Minibatch accuracy: 12.5%
Validation accuracy: 12.0%
Minibatch loss at step 50: 1.974358
Minibatch accuracy: 31.2%
Validation accuracy: 44.1%
Minibatch loss at step 100: 1.165790
Minibatch accuracy: 50.0%
Validation accuracy: 66.0%
Minibatch loss at step 150: 0.422755
Minibatch accuracy: 87.5%
Validation accuracy: 74.1%
Minibatch loss at step 200: 0.940693
Minibatch accuracy: 75.0%
Validation accuracy: 76.2%
Minibatch loss at step 250: 1.671060
Minibatch accuracy: 62.5%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.410579
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.531265
Minibatch accuracy: 93.8%
Validation accuracy: 76.2%
Minibatch loss at step 400: 0.279567
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.937390
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 500: 0.746215
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
M

## Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation(nn.max_pool()) of stride 2 and kernel size 2.

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 =  tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) 
        return tf.matmul(hidden, layer4_weights) + layer4_biases
        
    #  Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.538488
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.068700
Minibatch accuracy: 18.8%
Validation accuracy: 39.9%
Minibatch loss at step 100: 1.338221
Minibatch accuracy: 50.0%
Validation accuracy: 55.9%
Minibatch loss at step 150: 0.457582
Minibatch accuracy: 87.5%
Validation accuracy: 73.0%
Minibatch loss at step 200: 0.913617
Minibatch accuracy: 75.0%
Validation accuracy: 76.3%
Minibatch loss at step 250: 1.326888
Minibatch accuracy: 62.5%
Validation accuracy: 76.3%
Minibatch loss at step 300: 0.297368
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.410600
Minibatch accuracy: 93.8%
Validation accuracy: 78.1%
Minibatch loss at step 400: 0.232961
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.769053
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.693156
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
M

Validation accuracy: 87.4%
Minibatch loss at step 4550: 0.263690
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 4600: 0.799586
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 4650: 0.925504
Minibatch accuracy: 81.2%
Validation accuracy: 87.0%
Minibatch loss at step 4700: 0.487271
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 4750: 0.795111
Minibatch accuracy: 68.8%
Validation accuracy: 87.3%
Minibatch loss at step 4800: 0.298150
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 4850: 0.302727
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
Minibatch loss at step 4900: 0.099438
Minibatch accuracy: 93.8%
Validation accuracy: 87.7%
Minibatch loss at step 4950: 0.213980
Minibatch accuracy: 100.0%
Validation accuracy: 88.0%
Minibatch loss at step 5000: 1.067035
Minibatch accuracy: 68.8%
Validation accuracy: 87.6%
Minibatch loss at step 5050: 0.235199
Minibatch accuracy: 93.8

Validation accuracy: 89.0%
Minibatch loss at step 9050: 0.296754
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 9100: 0.236736
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 9150: 0.699378
Minibatch accuracy: 68.8%
Validation accuracy: 88.3%
Minibatch loss at step 9200: 0.323539
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 9250: 0.491961
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 9300: 0.939996
Minibatch accuracy: 81.2%
Validation accuracy: 88.6%
Minibatch loss at step 9350: 0.325015
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 9400: 0.372517
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 9450: 0.366636
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 9500: 0.142547
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 9550: 0.248135
Minibatch accuracy: 87.5%

Minibatch loss at step 13500: 0.276584
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 13550: 0.352375
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Minibatch loss at step 13600: 0.298182
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 13650: 0.492969
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 13700: 0.269469
Minibatch accuracy: 93.8%
Validation accuracy: 89.5%
Minibatch loss at step 13750: 0.788417
Minibatch accuracy: 75.0%
Validation accuracy: 89.2%
Minibatch loss at step 13800: 0.052255
Minibatch accuracy: 100.0%
Validation accuracy: 89.1%
Minibatch loss at step 13850: 0.085626
Minibatch accuracy: 100.0%
Validation accuracy: 89.2%
Minibatch loss at step 13900: 0.183266
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 13950: 0.372052
Minibatch accuracy: 81.2%
Validation accuracy: 89.3%
Minibatch loss at step 14000: 0.054540
Minibatch accuracy: 100.0%
Validation a

Minibatch loss at step 17950: 0.411931
Minibatch accuracy: 81.2%
Validation accuracy: 89.6%
Minibatch loss at step 18000: 0.365325
Minibatch accuracy: 87.5%
Validation accuracy: 89.8%
Minibatch loss at step 18050: 0.326597
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 18100: 0.480594
Minibatch accuracy: 87.5%
Validation accuracy: 89.9%
Minibatch loss at step 18150: 0.170843
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 18200: 0.493692
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 18250: 0.410786
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 18300: 0.099044
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 18350: 0.265716
Minibatch accuracy: 87.5%
Validation accuracy: 89.7%
Minibatch loss at step 18400: 0.609631
Minibatch accuracy: 81.2%
Validation accuracy: 89.8%
Minibatch loss at step 18450: 0.222901
Minibatch accuracy: 93.8%
Validation accu

By adding max pool layers into the model, the performance got significantly improved. I have an 95% test accuracy. However, the computational time increased exponentially too. It's longer than what I expected.

## Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.